# Capstone Project - The Battle of the Neighborhoods

![Image of Toronto](https://www.seetorontonow.com/wp-content/uploads/2020/06/Toronto-Skyline-view-from-lake.jpg)

## Table of Contents
- **[Introduction](#Introduction)**
- **[Business Problem](#Business)**
- **[Target Audience](#Target)**
- **[Data](#Data)**


<a id='Introduction'></a>
## Introduction
Toronto is the capital city of the Canadian province of Ontario. With a recorded population of 2,731,571 in 2016 it is the most populous city in Canada and the fourth most populous city in North America. Toronto is an international centre of business, finance, arts, and culture, and is recognized as one of the most multicultural and cosmopolitan cities in the world.

Toronto encompasses a geographical area formerly administered by many separate municipalities. These municipalities have each developed a distinct history and identity over the years, and their names remain in common use among Torontonians. Throughout the city there exist hundreds of small neighbourhoods and some larger neighbourhoods covering a few square kilometres.

Having such a diverse mix of people and culture, there exists tough competition the restaurant business in Toronto. Therefore it becomes a challenging task for hoteliers to decide the optimal location to start a new restaurant in a specific neighborhood.

<a id='Business'></a>
## Business Problem
This project uses data science methodologies to find an optimal location for starting a new restaurant in Toronto, Canada.
It tries to identify the most suitable neighborhood, by taking into consideration the competitors and which income group of people will be attracted most to it based on the population of the neighbourhood. It also considers other factors such as cuisine provided by other restaurants, proximity to the city center etc.

In summary, this project aims at analysing the restaurants and their cuisines in various neighborhoods of Toronto and figure out the best possible neighborhood to start a new restaurant

<a id='Target'></a>
## Target Audience
This project is beneficial to anyone who is planning to start a new restaurant business in Toronto. The project can also benefit already running restaurants to shift to some other cuisine for better prospects

<a id=’Data’></a>
## Data
The data requirements for this project are as follows:

- All the existing restaurants in the neighborhood along with their type and cuisine: obtained using the FourSquare API
- The age group of people in the neighborhood along with their income
- The distance of the neighborhood from the city center
- The postalcode details of neighboroods shall be obtained from //en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M